In [1]:
import requests
import json
import pandas as pd


parameters = {
   "beginDate" : "01/01/2014",
   "endDate" : "15/01/2024"}

parameters2 = {"granularity": "annual"}

url = "https://ec.europa.eu/agrifood/api/oliveOil/prices?"
url2 = "https://ec.europa.eu/agrifood/api/oliveOil/production?"

response = requests.get(url, params=parameters)
response.json()
df = pd.DataFrame.from_dict(response.json())


response2 = requests.get(url2, params=parameters2)
response2.json()
df2 = pd.DataFrame.from_dict(response2.json())

In [2]:
df = df.rename(columns={'memberStateName': 'country', 'beginDate': 'date', 'marketingYear': 'campaign'})
columns_to_remove = ['endDate']
df.drop(columns_to_remove, axis=1, inplace=True)
df = df.drop(["memberStateCode", "market", "weekNumber"], axis=1)

In [3]:
df2 = df2.rename(columns={'memberStateName': 'country', 'marketingYear': 'campaign', 'yearProductionQuantity':'production'})
columns_to_remove = ['memberStateCode','productionYear', 'endingStockQuantity']
df2.drop(columns_to_remove, axis=1, inplace=True)

In [4]:
to_drop=df2[df2["country"]=="Cyprus"].index
df2.drop(index=to_drop,inplace=True)
df2["country"].value_counts()

country
Croatia     12
France      12
Greece      12
Italy       12
Slovenia    12
Portugal    11
Spain       11
Malta        4
Name: count, dtype: int64

In [5]:
#1. What are the total olive oil price and production over the past 10 years?
#(price and time)

df2_production = df2.groupby("country").agg({"production":"sum"})
df2_production

,production
country,
Croatia,44.22
France,49.08
Greece,3133.50
Italy,3850.67
Malta,0.14
Portugal,1142.25
Slovenia,5.83
Spain,13786.11


In [6]:
df["price"] = df["price"].replace("[\€,]", "", regex=True).astype(float)
df_price = df.groupby(["country"]).agg({"price":sum})
df_price

,price
country,
Croatia,599342.30
France,210898.00
Greece,1090204.52
Italy,3944954.00
Malta,363900.00
Portugal,183892.80
Slovenia,90548.00
Spain,4530169.31


In [7]:
df3 = pd.merge(df_price, df2_production, on="country", how= "inner")
df3

,price,production
country,,
Croatia,599342.30,44.22
France,210898.00,49.08
Greece,1090204.52,3133.50
Italy,3944954.00,3850.67
Malta,363900.00,0.14
Portugal,183892.80,1142.25
Slovenia,90548.00,5.83
Spain,4530169.31,13786.11


In [8]:
#2.Which Member States are the major producers of olive oil?
#(countries and production)

df_max = df2.groupby(['country', 'campaign']).agg({'production':'max'})
df_max.sort_values(by= 'production', axis=0, ascending=False)

production
country campaign            
Spain   2018/19      1789.90
        2013/14      1775.80
        2021/22      1491.46
        2015/16      1397.90
        2020/21      1389.00
...                      ...
Malta   2013/14         0.04
        2012/13         0.04
        2015/16         0.04
        2014/15         0.02
France  2013/14         0.00

[86 rows x 1 columns]

In [9]:
#3. Which markets have the highest and lowest olive oil prices?(countries and prices)

df_sales = df.groupby(['country']).agg({'price':'max'})
df_sales.sort_values(by= 'price', axis=0, ascending=False)

,price
country,
Malta,2934.00
Slovenia,1845.00
Croatia,1566.89
France,1494.00
Italy,968.00
Spain,930.00
Greece,896.50
Portugal,880.00


In [10]:

#4. How has production evolved in these countries over time?
#(time, countries, production)

df_flut = df2.groupby(['campaign','country']).agg({'production':'sum'})
df_pivot = df2.pivot_table(index="country", columns= "campaign", values="production", aggfunc='sum')
df_pivot

campaign,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,2020/21,2021/22,2022/23,2023/24
country,,,,,,,,,,,,
Croatia,5.50,5.00,1.17,3.54,3.45,3.88,3.36,4.09,3.71,2.95,5.07,2.50
France,5.12,0.00,1.90,4.98,3.29,6.24,5.80,3.37,4.55,5.79,3.54,4.50
Greece,358.00,132.00,295.00,320.00,195.00,346.00,185.00,275.00,275.00,232.00,345.50,175.00
Italy,409.14,461.24,222.00,474.62,182.33,428.92,173.60,366.47,273.50,329.03,240.92,288.90
Malta,0.04,0.04,0.02,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Portugal,NaN,91.65,64.15,109.13,69.42,134.77,100.32,140.50,100.00,206.24,125.92,0.15
Slovenia,0.24,0.60,0.21,0.49,0.35,0.35,0.90,0.30,0.90,0.28,0.71,0.50
Spain,NaN,1775.80,837.60,1397.90,1286.60,1260.10,1789.90,1125.28,1389.00,1491.46,665.84,766.63


In [11]:
##########################################################################

In [12]:
#whats influences more the flutuation of prices and production - search for data and make a new df to get a decision to be shown
#----Law of Supply & Demand (this is key) - when the amount of product available is low, prices are high and when the product availability is high typically the prices are lower
#(production) by informa

#give result in percentage of how it will grow / prediction


